In [1]:
import numpy as np
import pandas as pd

In [2]:
data_path = "adult/adult.data"
test_path = "adult/adult.test"

column_names = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "sex",
    "capital_gain", "capital_loss", "hours_per_week", "native_country", "income"
]

df_train = pd.read_csv(data_path, header=None, names=column_names)
df_test = pd.read_csv(test_path, header=None, names=column_names)

df_train.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
df_train = df_train.apply(lambda x: x.str.strip() if x.dtype == "object" else x)   ### for detect " ?" and "? " and "?"
df_test  = df_test.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [4]:
df_train.replace('?', np.nan, inplace=True) 
df_test.replace('?', np.nan, inplace=True)

In [5]:
train_rows_with_nan = df_train.isna().any(axis=1).sum()
test_rows_with_nan = df_test.isna().any(axis=1).sum()
print(f"number of rows with 'NaN' in data : {train_rows_with_nan}")
print(f"number of rows with 'NaN' in test : {test_rows_with_nan}")

number of rows with 'NaN' in data : 2399
number of rows with 'NaN' in test : 1222


### Dropping NaN values due to the small number of rows containing NaN compared to the total dataset.

In [6]:
df_train.dropna(inplace=True)
train_rows_with_nan = df_train.isna().any(axis=1).sum()
df_test.dropna(inplace=True)
test_rows_with_nan = df_test.isna().any(axis=1).sum()
print(f"number of rows with 'NaN' in train : {train_rows_with_nan}")  
print(f"number of rows with 'NaN' in test : {test_rows_with_nan}")

number of rows with 'NaN' in train : 0
number of rows with 'NaN' in test : 0


In [7]:
print(f"Number of rows in data : {df_train.shape[0]}")
print(f"Number of rows in test : {df_test.shape[0]}")

Number of rows in data : 30162
Number of rows in test : 15060


In [8]:
df_train['income'] = df_train['income'].str.strip().str.replace('.', '', regex=False)   ### Remove trailing '.' in 'income' values to unify categories before OHE
df_test['income'] = df_test['income'].str.strip().str.replace('.', '', regex=False)

In [9]:
categorical_cols = [
    "workclass", "education", "marital_status", "occupation",
    "relationship", "race", "sex", "native_country", "income"
]

combined = pd.concat([df_train, df_test], axis=0)
combined_onehot = pd.get_dummies(combined, columns=categorical_cols)   #, drop_first=True)   for linear models

df_train_onehot = combined_onehot.iloc[:len(df_train), :].copy()
df_test_onehot = combined_onehot.iloc[len(df_train):, :].copy()

bool_cols_train = df_train_onehot.select_dtypes('bool').columns
df_train_onehot[bool_cols_train] = df_train_onehot[bool_cols_train].astype('int64')

bool_cols_test = df_test_onehot.select_dtypes('bool').columns
df_test_onehot[bool_cols_test] = df_test_onehot[bool_cols_test].astype('int64')


print(f"Train data shape : {df_train_onehot.shape}")
print(f"Test data shape : {df_test_onehot.shape}")

Train data shape : (30162, 106)
Test data shape : (15060, 106)


In [10]:
df_train_onehot

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia,income_<=50K,income_>50K
0,39,77516.0,13.0,2174.0,0.0,40.0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,50,83311.0,13.0,0.0,0.0,13.0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,38,215646.0,9.0,0.0,0.0,40.0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
3,53,234721.0,7.0,0.0,0.0,40.0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
4,28,338409.0,13.0,0.0,0.0,40.0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302.0,12.0,0.0,0.0,38.0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
32557,40,154374.0,9.0,0.0,0.0,40.0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
32558,58,151910.0,9.0,0.0,0.0,40.0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
32559,22,201490.0,9.0,0.0,0.0,20.0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0


In [11]:
for i, col in enumerate(df_train_onehot.columns, start=1):   # It shows there is no problem ...
    print(f"{i}: {col}")

1: age
2: fnlwgt
3: education_num
4: capital_gain
5: capital_loss
6: hours_per_week
7: workclass_Federal-gov
8: workclass_Local-gov
9: workclass_Private
10: workclass_Self-emp-inc
11: workclass_Self-emp-not-inc
12: workclass_State-gov
13: workclass_Without-pay
14: education_10th
15: education_11th
16: education_12th
17: education_1st-4th
18: education_5th-6th
19: education_7th-8th
20: education_9th
21: education_Assoc-acdm
22: education_Assoc-voc
23: education_Bachelors
24: education_Doctorate
25: education_HS-grad
26: education_Masters
27: education_Preschool
28: education_Prof-school
29: education_Some-college
30: marital_status_Divorced
31: marital_status_Married-AF-spouse
32: marital_status_Married-civ-spouse
33: marital_status_Married-spouse-absent
34: marital_status_Never-married
35: marital_status_Separated
36: marital_status_Widowed
37: occupation_Adm-clerical
38: occupation_Armed-Forces
39: occupation_Craft-repair
40: occupation_Exec-managerial
41: occupation_Farming-fishing
4